In [3]:
import torch
import torch.nn as nn
from torchvision import models

In [18]:

# Load pretrained ResNet model
resnet = models.resnet50(pretrained=True)


d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
list(resnet.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [15]:

# Remove the fully connected layer and the pooling layer
# Keep layers up to the penultimate layer
feature_extractor = nn.Sequential(*list(resnet.children())[:-2])


In [16]:
# Test the feature extractor
input_tensor = torch.randn(1, 3, 224, 224)  # Example input (batch_size=1, 3 channels, 224x224 image)
features = feature_extractor(input_tensor)
print("Feature shape:", features.shape)

Feature shape: torch.Size([1, 2048, 7, 7])


In [22]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from PIL import Image


In [23]:
input_folder = "D:/D-Documents/Self-Improvement/Python/Computer Vision/MS-TCN/Multi-Model/data/gtea/frames"
output_folder = "D:/D-Documents/Self-Improvement/Python/Computer Vision/MS-TCN/Multi-Model/data/gtea/extracted_frame_features"

In [24]:

# Create output directory if not exists
os.makedirs(output_folder, exist_ok=True)

# Load ResNet model for feature extraction
resnet = models.resnet50(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the fully connected layer
feature_extractor.eval()

# Define preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Process each video folder
for video_folder in os.listdir(input_folder):
    video_path = os.path.join(input_folder, video_folder)
    if not os.path.isdir(video_path):
        continue  # Skip if it's not a folder

    # Create a corresponding folder in the output directory
    video_output_path = os.path.join(output_folder, video_folder)
    os.makedirs(video_output_path, exist_ok=True)

    print(f"Processing video folder: {video_folder}")

    # Process each frame
    for frame_file in os.listdir(video_path):
        frame_path = os.path.join(video_path, frame_file)

        # Check for valid image file
        if not (frame_file.endswith(".png") or frame_file.endswith(".jpg")):
            continue

        # Load and preprocess frame
        image = Image.open(frame_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Extract features
        with torch.no_grad():
            features = feature_extractor(input_tensor).squeeze().numpy()  # Remove batch and spatial dimensions

        # Save features as .npy
        feature_file = os.path.splitext(frame_file)[0] + ".npy"
        feature_path = os.path.join(video_output_path, feature_file)
        np.save(feature_path, features)

    print(f"Features saved for video folder: {video_folder}")

print("Feature extraction completed!")


d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing video folder: S1_Cheese_C1
Features saved for video folder: S1_Cheese_C1
Processing video folder: S1_CofHoney_C1
Features saved for video folder: S1_CofHoney_C1
Processing video folder: S1_Hotdog_C1
Features saved for video folder: S1_Hotdog_C1
Processing video folder: S1_Pealate_C1
Features saved for video folder: S1_Pealate_C1
Processing video folder: S1_Peanut_C1
Features saved for video folder: S1_Peanut_C1
Processing video folder: S1_Tea_C1
Features saved for video folder: S1_Tea_C1
Processing video folder: S2_Cheese_C1
Features saved for video folder: S2_Cheese_C1
Processing video folder: S2_Coffee_C1
Features saved for video folder: S2_Coffee_C1
Processing video folder: S2_CofHoney_C1
Features saved for video folder: S2_CofHoney_C1
Processing video folder: S2_Hotdog_C1
Features saved for video folder: S2_Hotdog_C1
Processing video folder: S2_Pealate_C1
Features saved for video folder: S2_Pealate_C1
Processing video folder: S2_Peanut_C1
Features saved for video folder:

### For, Testing

In [30]:
import os
import pandas as pd


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



C:\Users\user\AppData\Roaming\Python\Python312\site-packages\IPython\core\completerlib.py:149: UserWarning: using rootmodules_cache requires you to install the `pickleshare` library.
  ip.db['rootmodules_cache'] = rootmodules_cache


In [26]:
dataset = "gtea"
gt_path = f"../data/{dataset}/action_labels/"
frames_path = f"../data/{dataset}/frames/"

In [31]:
for video_name in os.listdir(gt_path):

    if not video_name.endswith(".txt"):
        continue  

    # Ground truth label count
    with open(os.path.join(gt_path, video_name), 'r') as f:
        gt_labels = f.readlines()
    
    num_gt_labels = len(gt_labels)

    # For, testing
    print(f"Checking {video_name} - GT Labels: {num_gt_labels}")    

    # Frame count
    video_folder = os.path.join(frames_path, video_name.replace(".txt", ""))
    
    if not os.path.exists(video_folder):
        print(f"\nWarning: Frames not found for {video_name}\n")
    
        continue

    num_frames = len([f for f in os.listdir(video_folder) if f.endswith((".png", ".jpg"))])

    if num_gt_labels != num_frames:
        print(f"Mismatch for {video_name}: GT Labels = {num_gt_labels}, Frames = {num_frames}\n")

print("\nCheck completed!")

Checking S1_Cheese_C1.txt - GT Labels: 943
Mismatch for S1_Cheese_C1.txt: GT Labels = 943, Frames = 245

Checking S1_Coffee_C1.txt - GT Labels: 1178


Checking S1_CofHoney_C1.txt - GT Labels: 1235
Mismatch for S1_CofHoney_C1.txt: GT Labels = 1235, Frames = 250

Checking S1_Hotdog_C1.txt - GT Labels: 718
Mismatch for S1_Hotdog_C1.txt: GT Labels = 718, Frames = 250

Checking S1_Pealate_C1.txt - GT Labels: 1384
Mismatch for S1_Pealate_C1.txt: GT Labels = 1384, Frames = 249

Checking S1_Peanut_C1.txt - GT Labels: 1643
Mismatch for S1_Peanut_C1.txt: GT Labels = 1643, Frames = 250

Checking S1_Tea_C1.txt - GT Labels: 2009
Mismatch for S1_Tea_C1.txt: GT Labels = 2009, Frames = 250

Checking S2_Cheese_C1.txt - GT Labels: 634
Mismatch for S2_Cheese_C1.txt: GT Labels = 634, Frames = 169

Checking S2_Coffee_C1.txt - GT Labels: 1814
Mismatch for S2_Coffee_C1.txt: GT Labels = 1814, Frames = 212

Checking S2_CofHoney_C1.txt - GT Labels: 823
Mismatch for S2_CofHoney_C1.txt: GT Labels = 823, Frames = 

In [29]:
for i in range(1, 5):
    video_name = f"../data/{dataset}/action_labels/S{i}_Cheese_C1.txt"
    frame_path = f"../data/{dataset}/frames/S{i}_Cheese_C1"

    # Ground truth label count
    with open(video_name, 'r') as f:
        gt_labels = f.readlines()

    num_gt_labels = len(gt_labels)

    # For, testing
    print(f"Checking {video_name} - GT Labels: {num_gt_labels}")    

    if not os.path.exists(frame_path):
        print(f"\nWarning: Frames not found for {video_name}\n")

        continue

    num_frames = len([f for f in os.listdir(video_folder) if f.endswith((".png", ".jpg"))])

    if num_gt_labels != num_frames:
        print(f"Mismatch for {video_name}: GT Labels = {num_gt_labels}, Frames = {num_frames}\n")

print("\nCheck completed!")

Checking ../data/gtea/action_labels/S1_Cheese_C1.txt - GT Labels: 943
Mismatch for ../data/gtea/action_labels/S1_Cheese_C1.txt: GT Labels = 943, Frames = 250

Checking ../data/gtea/action_labels/S2_Cheese_C1.txt - GT Labels: 634
Mismatch for ../data/gtea/action_labels/S2_Cheese_C1.txt: GT Labels = 634, Frames = 250

Checking ../data/gtea/action_labels/S3_Cheese_C1.txt - GT Labels: 913
Mismatch for ../data/gtea/action_labels/S3_Cheese_C1.txt: GT Labels = 913, Frames = 250

Checking ../data/gtea/action_labels/S4_Cheese_C1.txt - GT Labels: 805
Mismatch for ../data/gtea/action_labels/S4_Cheese_C1.txt: GT Labels = 805, Frames = 250


Check completed!


In [33]:
dataset = "gtea"
extracted_xml_data_path = f"../data/{dataset}/extracted_xml_data/"

In [39]:
for extracted_xml_data in os.listdir(extracted_xml_data_path):
    xml_data = pd.read_csv(os.path.join(extracted_xml_data_path, extracted_xml_data))

    print(f"Checking {extracted_xml_data} - Rows: {xml_data.shape[0]}") # For, testing

    unique_values = xml_data[xml_data.columns[0]].unique()

    print(f"Number of unique values: {len(unique_values)}\n") # For, testing


Checking S1_Cheese_C1.csv - Rows: 9798
Number of unique values: 943

Checking S1_Coffee_C1.csv - Rows: 10986
Number of unique values: 1178

Checking S1_CofHoney_C1.csv - Rows: 11646
Number of unique values: 1235

Checking S1_Hotdog_C1.csv - Rows: 7386
Number of unique values: 718

Checking S1_Pealate_C1.csv - Rows: 12990
Number of unique values: 1384

Checking S1_Peanut_C1.csv - Rows: 14448
Number of unique values: 1643

Checking S1_Tea_C1.csv - Rows: 16248
Number of unique values: 2009

Checking S2_Cheese_C1.csv - Rows: 6684
Number of unique values: 634

Checking S2_Coffee_C1.csv - Rows: 17466
Number of unique values: 1814

Checking S2_CofHoney_C1.csv - Rows: 7884
Number of unique values: 823

Checking S2_Hotdog_C1.csv - Rows: 8826
Number of unique values: 811

Checking S2_Pealate_C1.csv - Rows: 12036
Number of unique values: 1181

Checking S2_Peanut_C1.csv - Rows: 14370
Number of unique values: 1465

Checking S2_Tea_C1.csv - Rows: 11628
Number of unique values: 1412

Checking S3_Chee